In [63]:
import numpy as np
import pandas as pd
from numpy.linalg import norm

## Necessary Functions

In [64]:
def standardize(X):
    if X.size == 0:
        raise ValueError("Input array X is empty.")
    mean = np.mean(X, axis=0)
    std_dev = np.std(X, axis=0)
    if np.any(std_dev == 0):
        raise ValueError("Standard deviation is zero for one or more features, cannot standardize.")
    X_std = (X - mean) / std_dev
    return X_std

In [65]:
def covariance_matrix(X):
    if X.size == 0:
        raise ValueError("Input array X is empty.")
    m = X.shape[0]
    return (1 / (m - 1)) * X.T @ X

In [66]:
def qr_decomposition_householder(A, max_iter=100, tol=1e-8):
    if isinstance(A, pd.DataFrame):
        A = A.values

    n, m = A.shape
    Q = np.eye(n)
    R = A.copy()

    for i in range(min(n, m, max_iter)):
        x = R[i:, i]
        if np.max(np.abs(x)) < tol:
            break
        e = np.zeros_like(x)
        e[0] = norm(x)
        u = x - e
        u /= norm(u)
        H = np.eye(n)
        H[i:, i:] -= 2.0 * np.outer(u, u)
        R = H @ R
        Q = Q @ H.T

    return Q, R

In [67]:
def eigen_decomp(A, max_iter=100, tol=1e-8):
    n = A.shape[0]
    A_k = A.copy()
    Q_total = np.eye(n)
    
    for _ in range(max_iter):
        Q, R = qr_decomposition_householder(A_k)
        A_k = R @ Q
        Q_total = Q_total @ Q
        
        if np.allclose(A_k - np.diag(np.diagonal(A_k)), 0, atol=tol):
            break

    eigenvalues = np.diagonal(A_k)
    eigenvectors = Q_total

    for i in range(n):
        eigenvectors[:, i] /= norm(eigenvectors[:, i])

    return eigenvalues, eigenvectors

In [68]:
def householder_reflection(a):
    """
    Compute the Householder vector for a given vector a.
    
    Args:
    a -- (m,) vector
    
    Returns:
    v -- (m,) Householder vector
    """
    norm_a = np.linalg.norm(a)
    sign = -np.sign(a[0]) if a[0] != 0 else -1
    v = a.copy()
    
    v[0] -= sign * norm_a
    v = v / np.linalg.norm(v)
    
    return v

In [69]:
def reduced_qr_decomposition_householder(A):
    """
    Perform reduced QR decomposition using Householder transformations.
    
    Args:
    A -- (m, n) matrix with m >= n
    
    Returns:
    Q -- (m, n) orthonormal matrix
    R -- (n, n) upper triangular matrix
    """
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    
    for i in range(n):
        a = R[i:, i]

        v = householder_reflection(a)
        
        H = np.eye(m)
        H[i:, i:] -= 2.0 * np.outer(v, v)
        
        R = H @ R
        
        Q = Q @ H
    
    return Q[:, :n], R[:n, :]

In [70]:
def reduced_qr_decomposition_householder_without_q(A, b):
    """
    Perform reduced QR decomposition using Householder transformations 
    without storing Q explicitly, and apply the transformations to vector b.
    
    Args:
    A -- (m, n) matrix with m >= n
    b -- (m,) vector
    
    Returns:
    R -- (n, n) upper triangular matrix
    b_hat -- (n,) transformed vector (Q^T b)
    """
    _, n = A.shape
    R = A.copy()
    b_hat = b.copy()

    for i in range(n):
        a = R[i:, i]

        v = householder_reflection(a)

        R[i:, i:] -= 2.0 * np.outer(v, v @ R[i:, i:])
        
        b_hat[i:] -= 2.0 * v @ (v @ b_hat[i:])
    
    return R[:n, :], b_hat[:n]

In [71]:
def pca(X, threshold):
    X_std = standardize(X)
    cov_matrix = covariance_matrix(X_std)
    eigenvalues, eigenvectors = eigen_decomp(cov_matrix)
    total_variance = np.sum(eigenvalues)
    variance_ratio = eigenvalues / total_variance
    cumulative_variance_ratio = np.cumsum(variance_ratio)
    print(f"Explained variance ratio: {cumulative_variance_ratio}")
    n_components = np.argmax(cumulative_variance_ratio >= threshold) + 1
    V_k  = eigenvectors[:, :n_components]
    Z = np.dot(X_std, V_k)

    return Z, V_k, n_components

In [72]:
def normal_equation(X, y):
    if X.size == 0 or y.size == 0:
        raise ValueError("Input arrays X or y are empty.")
    return np.linalg.inv(X.T @ X) @ X.T @ y

In [73]:
def qr_with_q_equation(X, y):
    if X.size == 0 or y.size == 0:
        raise ValueError("Input arrays X or y are empty.")
    Q, R = reduced_qr_decomposition_householder(X)
    return np.linalg.inv(R) @ Q.T @ y

In [74]:
def qr_without_q_equation(X, y):
    R, b_hat = reduced_qr_decomposition_householder_without_q(X, y)

    return np.linalg.inv(R) @ b_hat

In [75]:
def mean_square_error(y_true, y_pred):
    if y_true.size == 0 or y_pred.size == 0:
        raise ValueError("Input arrays y_true or y_pred are empty.")
    return np.mean((y_true - y_pred) ** 2)

In [76]:
def residual_error_norm(y, predictions):
    residuals = y - predictions
    norm = np.linalg.norm(residuals)
    return norm

In [77]:
def backwards_substitution(U, b):
    n = U.shape[0]
    x = np.zeros_like(b)

    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - U[i, i + 1:] @ x[i + 1:]) / U[i, i]

    return x

## Code Execution

### Open the dataset

In [78]:
df = pd.read_csv("pokindex_data.csv")
df

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,WinningPercentage
0,70,90,45,15,45,50,36.290323
1,40,27,60,37,50,66,36.220472
2,75,75,60,83,60,60,39.344262
3,85,115,80,105,80,50,30.630631
4,83,106,65,86,65,85,66.406250
...,...,...,...,...,...,...,...
195,50,65,64,44,48,43,21.969697
196,60,85,69,65,79,80,57.600000
197,45,50,43,40,38,62,40.441176
198,55,45,50,45,65,80,55.462185


In [79]:
X_df = df.iloc[:, :-1]
X_df

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,70,90,45,15,45,50
1,40,27,60,37,50,66
2,75,75,60,83,60,60
3,85,115,80,105,80,50
4,83,106,65,86,65,85
...,...,...,...,...,...,...
195,50,65,64,44,48,43
196,60,85,69,65,79,80
197,45,50,43,40,38,62
198,55,45,50,45,65,80


### Standardize the dataset

In [80]:
std_X = standardize(X_df)
std_X

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0.152909,0.405283,-0.922707,-1.788067,-1.045263,-0.649041
1,-1.180600,-1.596508,-0.433985,-1.110710,-0.852552,-0.094897
2,0.375161,-0.071334,-0.433985,0.305580,-0.467131,-0.302701
3,0.819664,1.199645,0.217644,0.982936,0.303712,-0.649041
4,0.730763,0.913675,-0.271078,0.397947,-0.274420,0.563148
...,...,...,...,...,...,...
195,-0.736097,-0.389078,-0.303659,-0.895188,-0.929636,-0.891478
196,-0.291594,0.246411,-0.140752,-0.248621,0.265170,0.389979
197,-0.958349,-0.865695,-0.987870,-1.018344,-1.315058,-0.233433
198,-0.513846,-1.024568,-0.759800,-0.864399,-0.274420,0.389979


### Get the covariance matrix

In [81]:
cov_matrix = covariance_matrix(std_X)
cov_matrix

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
HP,1.005025,0.555958,0.407711,0.423108,0.414206,0.250041
Attack,0.555958,1.005025,0.492852,0.450232,0.288810,0.392548
Defense,0.407711,0.492852,1.005025,0.265119,0.592316,-0.052182
Sp. Atk,0.423108,0.450232,0.265119,1.005025,0.527509,0.506228
Sp. Def,0.414206,0.288810,0.592316,0.527509,1.005025,0.193899
Speed,0.250041,0.392548,-0.052182,0.506228,0.193899,1.005025


In [82]:
Q, R = qr_decomposition_householder(cov_matrix)
print(Q)
print(R)

[[ 0.72940987 -0.54500747 -0.24840545 -0.30687205 -0.12216702  0.01129624]
 [ 0.40349392  0.75994714 -0.16466594 -0.32524921  0.14040878 -0.32719482]
 [ 0.29590123  0.18570912  0.78317796 -0.10122822 -0.36807509  0.34476142]
 [ 0.3070758   0.09953746 -0.10013326  0.78853923 -0.41568206 -0.30196751]
 [ 0.30061505 -0.13379389  0.38231174  0.32291456  0.79276163 -0.11323736]
 [ 0.18147012  0.25129826 -0.37632858  0.25244288  0.16917076  0.81850378]]
[[ 1.37786061e+00  1.25319031e+00  1.04364078e+00  1.12779520e+00
   1.09322398e+00  7.21453556e-01]
 [ 1.05901374e-16  6.57112083e-01  2.73006452e-01  3.17464948e-01
   7.05012515e-02  4.29358093e-01]
 [-1.29706269e-17 -1.67268141e-17  8.24219748e-01 -6.10760271e-02
   5.71882787e-01 -5.22398205e-01]
 [ 7.99218584e-17 -3.67751733e-17 -9.16170639e-18  7.87520880e-01
   5.08444916e-01  5.16381194e-01]
 [ 3.54139474e-17  1.86305448e-18 -5.77879182e-17 -6.92280869e-18
   3.82203678e-01  1.57084194e-01]
 [-2.31351849e-17 -1.20753692e-17  5.2799088

In [83]:
eigenvalues, eigenvectors = eigen_decomp(cov_matrix)

print(eigenvalues)
print(eigenvectors)

[2.94914287 1.20231921 0.77529324 0.51470719 0.37558598 0.21310226]
[[ 0.43354602 -0.06079581  0.39576925 -0.77440082  0.17656095  0.14435383]
 [ 0.44686717  0.06341416  0.57662133  0.39600225 -0.21287504 -0.51152972]
 [ 0.3863902  -0.57758994  0.03447058  0.41114815  0.10449576  0.57960512]
 [ 0.44243289  0.30046001 -0.38034996 -0.09211424 -0.71540397  0.22141129]
 [ 0.42577579 -0.27598307 -0.6008831  -0.07390754  0.31323149 -0.52717315]
 [ 0.29328878  0.7015907  -0.06292642  0.2459582   0.55015842  0.23371506]]


### Get the Z dataset, transformation matrix, and optimal k for PCA

In [84]:
Z, V_k, n_components = pca(X_df, 0.90)

principal_components = pd.DataFrame(data=Z, columns=[f"PC{i+1}" for i in range(n_components)])
principal_components

Explained variance ratio: [0.48906619 0.6884508  0.81702026 0.90237587 0.96466054 1.        ]


,PC1,PC2,PC3,PC4
0,-1.535627,-0.154778,1.611419,-0.254967
1,-2.275202,0.056187,-0.462071,0.245586
2,-0.189388,0.231696,0.275896,-0.565281
3,1.349380,-0.343314,0.508127,-0.342828
4,0.844757,0.760486,0.784812,-0.193403
...,...,...,...,...
195,-1.663667,-0.442388,0.429042,0.223010
196,0.046590,0.240373,-0.067483,0.364742
197,-2.262974,0.467138,0.279696,0.126750
198,-1.359104,0.494743,-0.351214,-0.124375


In [85]:
V_k

array([[ 0.43354602, -0.06079581,  0.39576925, -0.77440082],
       [ 0.44686717,  0.06341416,  0.57662133,  0.39600225],
       [ 0.3863902 , -0.57758994,  0.03447058,  0.41114815],
       [ 0.44243289,  0.30046001, -0.38034996, -0.09211424],
       [ 0.42577579, -0.27598307, -0.6008831 , -0.07390754],
       [ 0.29328878,  0.7015907 , -0.06292642,  0.2459582 ]])

In [86]:
n_components

np.int64(4)

## Linear Regression

In [87]:
y = df.iloc[:, -1]
y

0      36.290323
1      36.220472
2      39.344262
3      30.630631
4      66.406250
         ...    
195    21.969697
196    57.600000
197    40.441176
198    55.462185
199    60.800000
Name: WinningPercentage, Length: 200, dtype: float64

### Normal equation

In [88]:
X = np.c_[np.ones(principal_components.shape[0]), principal_components]

theta_normal = normal_equation(X, y)

y_pred_normal = pd.Series(np.dot(X, theta_normal))
y_pred_normal

0      35.029023
1      32.274892
2      48.278115
3      54.918159
4      68.328011
         ...    
195    31.295163
196    56.409055
197    38.697193
198    44.247542
199    55.378288
Length: 200, dtype: float64

In [89]:
normal_error = residual_error_norm(y, y_pred_normal)
normal_error

np.float64(151.0593158622192)

### Stored Q QR decomposition (Householder transformation)

In [90]:
theta_qr_with_q = qr_with_q_equation(X, y)
theta_qr_with_q

array([50.01524827,  8.62091413, 14.78978116,  1.4352154 ,  6.94724482])

In [91]:
y_pred_qr_with_q = pd.Series(np.dot(X, theta_qr_with_q))
y_pred_qr_with_q

0      35.029023
1      32.274892
2      48.278115
3      54.918159
4      68.328011
         ...    
195    31.295163
196    56.409055
197    38.697193
198    44.247542
199    55.378288
Length: 200, dtype: float64

In [92]:
qr_with_q_error = residual_error_norm(y, y_pred_qr_with_q)
qr_with_q_error

np.float64(151.0593158622192)

### Not stored Q QR decomposition (Householder transformation)

In [95]:
theta_qr_without_q = qr_without_q_equation(X, y)
theta_qr_without_q

array([-707.32242427,  208.84677473, -228.76953194,  -17.82693172,
         70.31034951])

In [96]:
y_pred_qr_without_q = pd.Series(np.dot(X, theta_qr_without_q))
y_pred_qr_without_q

0     -1039.278242
1     -1169.840287
2      -844.543981
3      -380.131783
4      -732.462732
          ...     
195    -945.537611
196    -725.734146
197   -1282.878409
198   -1106.832940
199    -284.599247
Length: 200, dtype: float64

In [98]:
error_theta_qr_without_q = residual_error_norm(y, y_pred_qr_without_q)
error_theta_qr_without_q

np.float64(12366.279073764219)

### Data without outlier using the QR decomposition

In [101]:
no_outlier_df = pd.read_csv("pokindex_data_nooutlier.csv")
no_outlier_df

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,WinningPercentage
0,70,90,45,15,45,50,36.290323
1,40,27,60,37,50,66,36.220472
2,75,75,60,83,60,60,39.344262
3,85,115,80,105,80,50,30.630631
4,83,106,65,86,65,85,66.406250
...,...,...,...,...,...,...,...
179,50,65,64,44,48,43,21.969697
180,60,85,69,65,79,80,57.600000
181,45,50,43,40,38,62,40.441176
182,55,45,50,45,65,80,55.462185


In [102]:
X_no_outlier_df = no_outlier_df.iloc[:, :-1]
X_no_outlier_df

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,70,90,45,15,45,50
1,40,27,60,37,50,66
2,75,75,60,83,60,60
3,85,115,80,105,80,50
4,83,106,65,86,65,85
...,...,...,...,...,...,...
179,50,65,64,44,48,43
180,60,85,69,65,79,80
181,45,50,43,40,38,62
182,55,45,50,45,65,80


In [103]:
X_no_outlier_std = standardize(X_no_outlier_df)
X_no_outlier_std

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0.281459,0.524904,-0.942009,-1.831199,-1.030554,-0.641841
1,-1.255290,-1.658336,-0.362828,-1.102321,-0.819675,-0.058457
2,0.537584,0.005085,-0.362828,0.421698,-0.397918,-0.277226
3,1.049834,1.391270,0.409414,1.150577,0.445595,-0.641841
4,0.947384,1.079378,-0.169767,0.521091,-0.187040,0.634311
...,...,...,...,...,...,...
179,-0.743041,-0.341461,-0.208379,-0.870405,-0.904027,-0.897072
180,-0.230791,0.351631,-0.015319,-0.174657,0.403420,0.452004
181,-0.999165,-0.861280,-1.019233,-1.002928,-1.325783,-0.204303
182,-0.486916,-1.034553,-0.748948,-0.837274,-0.187040,0.452004


In [104]:
cov_matrix_no_outlier = covariance_matrix(X_no_outlier_std)
cov_matrix_no_outlier

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
HP,1.005464,0.577163,0.469326,0.419522,0.493828,0.263993
Attack,0.577163,1.005464,0.536234,0.396956,0.299295,0.377378
Defense,0.469326,0.536234,1.005464,0.226107,0.569609,0.003423
Sp. Atk,0.419522,0.396956,0.226107,1.005464,0.557707,0.504540
Sp. Def,0.493828,0.299295,0.569609,0.557707,1.005464,0.247565
Speed,0.263993,0.377378,0.003423,0.504540,0.247565,1.005464


In [106]:
eigenvalues_no_outlier, eigenvectors_no_outlier = eigen_decomp(cov_matrix_no_outlier)

print(eigenvalues_no_outlier)
print(eigenvectors_no_outlier)

[3.02292626 1.15561446 0.78339176 0.48381804 0.3862541  0.20078226]
[[ 4.48805594e-01 -1.41864152e-01  2.09458644e-01 -7.94845686e-01
   2.60997607e-01  1.86214289e-01]
 [ 4.35537463e-01 -6.25359472e-02  6.34115270e-01  1.96910903e-01
  -3.31358654e-01 -5.05689368e-01]
 [ 3.91057722e-01 -5.68664275e-01 -7.93464637e-04  4.73103254e-01
   7.26372375e-02  5.42762668e-01]
 [ 4.18565970e-01  3.96964865e-01 -3.53707329e-01 -8.46728990e-02
  -6.76443227e-01  2.78150338e-01]
 [ 4.37304900e-01 -1.35865201e-01 -6.29906415e-01  6.80385894e-02
   2.83274157e-01 -5.55562312e-01]
 [ 2.99201384e-01  6.90321884e-01  1.79257648e-01  3.06302128e-01
   5.28191098e-01  1.70276525e-01]]


In [107]:
Z_no_outlier, V_k_no_outlier, n_components_no_outlier = pca(X_no_outlier_df, 0.90)
print(V_k_no_outlier)
print(n_components_no_outlier)

Explained variance ratio: [0.50108289 0.69263854 0.82249424 0.90269234 0.96671816 1.        ]
[[ 4.48805594e-01 -1.41864152e-01  2.09458644e-01 -7.94845686e-01]
 [ 4.35537463e-01 -6.25359472e-02  6.34115270e-01  1.96910903e-01]
 [ 3.91057722e-01 -5.68664275e-01 -7.93464637e-04  4.73103254e-01]
 [ 4.18565970e-01  3.96964865e-01 -3.53707329e-01 -8.46728990e-02]
 [ 4.37304900e-01 -1.35865201e-01 -6.29906415e-01  6.80385894e-02]
 [ 2.99201384e-01  6.90321884e-01  1.79257648e-01  3.06302128e-01]]
4


In [110]:
pca_no_outlier = pd.DataFrame(data=Z_no_outlier, columns=[f"PC{i+1}" for i in range(n_components_no_outlier)])
pca_no_outlier

,PC1,PC2,PC3,PC4
0,-1.422628,-0.567050,1.574357,-0.677686
1,-2.264868,0.121542,-0.418481,0.519224
2,0.021149,0.159832,0.167913,-0.745645
3,1.721639,-0.515637,0.299092,-0.630507
4,1.155016,0.564787,0.930232,-0.483357
...,...,...,...,...
179,-1.591750,-0.596700,0.344517,0.162196
180,0.282130,0.207347,0.063332,0.426122
181,-2.282821,0.416174,0.398614,0.074522
182,-1.259007,0.564745,-0.262428,0.025596


In [116]:
X_no_outlier = np.c_[np.ones(pca_no_outlier.shape[0]), pca_no_outlier]
y_no_outlier = no_outlier_df.iloc[:, -1]

print(y_no_outlier)
print(X_no_outlier)


0      36.290323
1      36.220472
2      39.344262
3      30.630631
4      66.406250
         ...    
179    21.969697
180    57.600000
181    40.441176
182    55.462185
183    60.800000
Name: WinningPercentage, Length: 184, dtype: float64
[[ 1.00000000e+00 -1.42262756e+00 -5.67050201e-01  1.57435735e+00
  -6.77686230e-01]
 [ 1.00000000e+00 -2.26486787e+00  1.21541758e-01 -4.18481157e-01
   5.19224301e-01]
 [ 1.00000000e+00  2.11492128e-02  1.59832373e-01  1.67912797e-01
  -7.45645002e-01]
 [ 1.00000000e+00  1.72163914e+00 -5.15636666e-01  2.99091601e-01
  -6.30506883e-01]
 [ 1.00000000e+00  1.15501603e+00  5.64786598e-01  9.30231859e-01
  -4.83357246e-01]
 [ 1.00000000e+00  3.50393409e-01 -8.73515262e-01  3.06030693e-01
  -1.71766874e+00]
 [ 1.00000000e+00  1.02045758e+00 -8.88399142e-01  9.52553611e-03
   1.01144328e+00]
 [ 1.00000000e+00  6.42387244e-01 -1.13643597e+00 -3.62379674e-01
   1.43790834e+00]
 [ 1.00000000e+00 -4.51907014e-01 -1.32921715e+00 -5.73564985e-01
   4.83750152e

In [117]:
theta_qr_with_q_no_outlier = qr_with_q_equation(X_no_outlier, y_no_outlier)
theta_qr_with_q_no_outlier

array([49.30751232,  8.69087917, 13.77761601,  6.40859288,  8.24078859])

In [118]:
y_pred_qr_with_q_no_outlier = pd.Series(np.dot(X_no_outlier, theta_qr_with_q_no_outlier))
y_pred_qr_with_q_no_outlier

0      33.635775
1      32.895317
2      46.624809
3      53.886708
4      69.105262
         ...    
179    30.797189
180    58.533670
181    38.370348
182    44.675612
183    58.907385
Length: 184, dtype: float64

In [119]:
error_qr_without_outlier = residual_error_norm(y_no_outlier, y_pred_qr_with_q_no_outlier)
error_qr_without_outlier

np.float64(129.57043054065807)